Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) 
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # CONV weights and biases Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # FC weights and biases Variables.
    layer3_weights = tf.Variable(tf.truncated_normal([(image_size // 4) * (image_size // 4) * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    print('')

Initialized
Minibatch loss at step 0: 3.446203
Minibatch accuracy: 6.2%
Validation accuracy: 10.3%
Minibatch loss at step 50: 1.428672
Minibatch accuracy: 37.5%
Validation accuracy: 62.2%
Minibatch loss at step 100: 0.456124
Minibatch accuracy: 93.8%
Validation accuracy: 75.6%
Minibatch loss at step 150: 0.492503
Minibatch accuracy: 87.5%
Validation accuracy: 76.2%
Minibatch loss at step 200: 0.366230
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 250: 1.389549
Minibatch accuracy: 75.0%
Validation accuracy: 75.8%
Minibatch loss at step 300: 0.714889
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.911391
Minibatch accuracy: 68.8%
Validation accuracy: 76.5%
Minibatch loss at step 400: 0.937076
Minibatch accuracy: 81.2%
Validation accuracy: 76.4%
Minibatch loss at step 450: 0.619984
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 500: 0.634224
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

Ok, here we replace the two convolutional layers with 2 max pooling operation. But my question is: Does that make sense to reduce the dimensionality and imporve the test accuracy?

Just do it!

In [7]:
batch_size = 128
patch_size = 5
depth = 1 # There is just max pooling operation, and the input image is just 1 channel, so the depth is 1
num_hidden = 128

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # FC weights and biases Variables.
    layer3_weights = tf.Variable(tf.truncated_normal([(image_size // 4) * (image_size // 4) * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        pool_1 = tf.nn.max_pool(data, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        pool_2 = tf.nn.max_pool(pool_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    print('')

Initialized
Minibatch loss at step 0: 2.787420
Minibatch accuracy: 14.1%
Validation accuracy: 7.8%
Minibatch loss at step 50: 1.669891
Minibatch accuracy: 60.2%
Validation accuracy: 59.0%
Minibatch loss at step 100: 1.284554
Minibatch accuracy: 69.5%
Validation accuracy: 69.6%
Minibatch loss at step 150: 1.128852
Minibatch accuracy: 71.9%
Validation accuracy: 72.2%
Minibatch loss at step 200: 1.137437
Minibatch accuracy: 68.8%
Validation accuracy: 72.0%
Minibatch loss at step 250: 0.952196
Minibatch accuracy: 71.9%
Validation accuracy: 73.3%
Minibatch loss at step 300: 0.991356
Minibatch accuracy: 68.8%
Validation accuracy: 74.3%
Minibatch loss at step 350: 0.983892
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 400: 0.970151
Minibatch accuracy: 71.1%
Validation accuracy: 75.6%
Minibatch loss at step 450: 0.831835
Minibatch accuracy: 78.9%
Validation accuracy: 76.7%
Minibatch loss at step 500: 0.922619
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Before write the code, here show the LeNet's topology

![LetNet Topology](./LetNet.png)

In [14]:
batch_size = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Weights and Biases
    # conv_1 is 5*5 with no zero padding and stride 1
    conv_1_weights = tf.Variable(tf.truncated_normal([5, 5, num_channels, 20], stddev=0.1))
    conv_1_biases = tf.Variable(tf.zeros([20]))
    # conv_2 is 5*5 with no zero padding and stride 1
    conv_2_weights = tf.Variable(tf.truncated_normal([5, 5, 20, 50], stddev=0.1))
    conv_2_biases = tf.Variable(tf.zeros([50]))
    # FC input layer
    input_layer_weights = tf.Variable(tf.truncated_normal([50*4*4, 500], stddev=0.1))
    input_layer_biases = tf.Variable(tf.constant(1.0, shape=[500]))
    hidden_layer_weights = tf.Variable(tf.truncated_normal([500, 10], stddev=0.1))
    hidden_layer_biases = tf.Variable(tf.constant(1.0, shape=[10]))
    
    # Model.
    def LeNet(data):
        conv_1 = tf.nn.conv2d(data, conv_1_weights, strides=[1, 1, 1, 1], padding='VALID') + conv_1_biases # add biases
        conv_1 = tf.nn.relu(conv_1)
        pool_1 = tf.nn.max_pool(conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv_2 = tf.nn.conv2d(pool_1, conv_2_weights, strides=[1, 1, 1, 1], padding='VALID') + conv_2_biases # add biases
        conv_2 = tf.nn.relu(conv_2)
        pool_2 = tf.nn.max_pool(conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        shapes = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shapes[0], shapes[1]*shapes[2]*shapes[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, input_layer_weights) + input_layer_biases)
        logits = tf.matmul(hidden, hidden_layer_weights) + hidden_layer_biases
        return logits
    
    # Training computation.
    logits = LeNet(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(learning_rate=0.1, global_step=global_step, decay_steps=3000, decay_rate=0.9, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(LeNet(tf_valid_dataset))
    test_prediction = tf.nn.softmax(LeNet(tf_test_dataset))

In [15]:
num_steps = 10000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    print('')

Initialized
Minibatch loss at step 0: 3.710216
Minibatch accuracy: 8.6%
Validation accuracy: 15.1%
Minibatch loss at step 1000: 0.276012
Minibatch accuracy: 91.4%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 0.221058
Minibatch accuracy: 92.2%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.242570
Minibatch accuracy: 93.0%
Validation accuracy: 89.6%
Minibatch loss at step 4000: 0.352597
Minibatch accuracy: 89.1%
Validation accuracy: 90.2%
Minibatch loss at step 5000: 0.171234
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 6000: 0.303108
Minibatch accuracy: 91.4%
Validation accuracy: 90.5%
Minibatch loss at step 7000: 0.353942
Minibatch accuracy: 90.6%
Validation accuracy: 90.9%
Minibatch loss at step 8000: 0.175381
Minibatch accuracy: 96.1%
Validation accuracy: 90.4%
Minibatch loss at step 9000: 0.250350
Minibatch accuracy: 92.2%
Validation accuracy: 90.9%
Test accuracy: 95.9%

